In [2]:
import pandas as pd # for data manipulation 
import numpy as np
import networkx as nx # for drawing graphs
import matplotlib.pyplot as plt # for drawing graphs
import os, sys, glob, math
import time

def rssi_to_np(rssi):
    # Function to convert rssi data from string (e.g. "435 pW") to exp (435e-12)
    rssi_num = np.zeros(rssi.shape)
    index = 0
    for r in rssi:
        num = r[0:-2]
        expn = r[-2:]
        # print(num)
        # print(expn)
        if expn == " W":
            # print(num)
            # print(index)
            rssi_num[index] = float(num)
        elif expn == "mW":
            rssi_num[index] = float(num) * 1e-3
        elif expn == "uW":
            rssi_num[index] = float(num) * 1e-6
        elif expn == "nW":
            rssi_num[index] = float(num) * 1e-9
        elif expn == "pW":
            rssi_num[index] = float(num) * 1e-12
        else:
            print(expn)
            raise ValueError("Unhandled unit prefix")
        index += 1
    return rssi_num

def compile_micro_sim_data(file_list):
    '''
    VERSION 2
    Function to compile data from the CSV files generated by each micro-simulation
    Update: To specifically return the rx_df, tx_df, mon_df and pd_df in lists, so that specific dfs can be accessed (instead of aggregating UAV dfs)
    Input: file_list - List of simulation files belonging to a certain scenario (micro-sim)
    Output: concatenates the raw data to UL and DL dataframes
    '''

    # Let's get the GCS dfs ===============================================================
    gcs_rx_file = [file for file in file_list if (('_GCS-' in file) and ('-Rx' in file))]
    gcs_tx_file = [file for file in file_list if (('_GCS-' in file) and ('-Tx' in file))]
    gcs_mon_file = [file for file in file_list if (('_GCS-' in file) and ('Wlan' in file))]
    gcs_pd_file = [file for file in file_list if (('_GCS-' in file) and ('PacketDrop' in file))]
    if len(gcs_rx_file) > 0:
        gcs_rx_df = pd.read_csv(gcs_rx_file[0])
    else:
        print("GCS RX File Missing")
        print(file_list[0])
    if len(gcs_tx_file) > 0:
        gcs_tx_df = pd.read_csv(gcs_tx_file[0])
    else:
        print("GCS TX File Missing")
        print(file_list[0])
    if len(gcs_pd_file) > 0:
        gcs_pd_df = pd.read_csv(gcs_pd_file[0])
    else:
        print("GCS PD File Missing")
        print(file_list[0])
    if len(gcs_mon_file) > 0:
        gcs_mon_df = pd.read_csv(gcs_mon_file[0]) # Mon file is optional
        gcs_mon_df["Addr"] = "192.168.0.1"
    else: 
        gcs_mon_df = None

    # Let's get the GW dfs ===============================================================
    gw_rx_file = [file for file in file_list if (('_GW-' in file) and ('-Rx' in file))]
    gw_tx_file = [file for file in file_list if (('_GW-' in file) and ('-Tx' in file))]
    gw_mon_file = [file for file in file_list if (('_GW-' in file) and ('Wlan' in file))]
    gw_pd_file = [file for file in file_list if (('_GW-' in file) and ('PacketDrop' in file))]
    if len(gw_rx_file) > 0:
        gw_rx_df = pd.read_csv(gw_rx_file[0])
    else:
        print("GW RX File Missing")
        print(file_list[0])
    if len(gw_tx_file) > 0:
        gw_tx_df = pd.read_csv(gw_tx_file[0])
    else:
        print("GW TX File Missing")
        print(file_list[0])
    if len(gw_pd_file) > 0:
        gw_pd_df = pd.read_csv(gw_pd_file[0])
    else:
        print("GW PD File Missing")
        print(file_list[0])
    if len(gw_mon_file) > 0:
        gw_mon_df = pd.read_csv(gw_mon_file[0]) # Mon file is optional
        gw_mon_df["Addr"] = "192.168.0.2"
    else:
        gw_mon_df = None

    # Let's get the UAVs dfs ===============================================================
    uavs_rx_df_list = []
    uavs_tx_df_list = []
    uavs_mon_df_list = []
    uavs_pd_df_list = []
    uav_rx_files = [file for file in file_list if (('_UAV-' in file) and ('-Rx' in file))]
    uav_tx_files = [file for file in file_list if (('_UAV-' in file) and ('-Tx' in file))]
    uav_mon_files = [file for file in file_list if (('_UAV-' in file) and ('Wlan' in file))]
    uav_pd_files = [file for file in file_list if (('_UAV-' in file) and ('PacketDrop' in file))]
    uav_rx_files.sort()
    uav_tx_files.sort()
    uav_mon_files.sort()
    uav_pd_files.sort()
    if len(uav_rx_files) > 0:
        for uav_rx_file in uav_rx_files:
            uavs_rx_df_list.append(pd.read_csv(uav_rx_file))
    else:
        print("UAV RX File(s) Missing")
        print(file_list[0])
    if len(uav_tx_files) > 0:
        for uav_tx_file in uav_tx_files:
            uavs_tx_df_list.append(pd.read_csv(uav_tx_file))
    else:
        print("UAV TX File(s) Missing")
        print(file_list[0])
    if len(uav_pd_files) > 0:
        for uav_pd_file in uav_pd_files:
            uavs_pd_df_list.append(pd.read_csv(uav_pd_file))
    else:
        print("UAV PD File(s) Missing")
        print(file_list[0])
    if len(uav_mon_files) > 0: # UAV mon files are optional now
        uav_member_index = 3
        for uav_mon_file in uav_mon_files:
            uav_mon_df = pd.read_csv(uav_mon_file)
            uav_mon_df["Addr"] = "192.168.0." + str(uav_member_index)
            uavs_mon_df_list.append(uav_mon_df)
            uav_member_index += 1
    else:
        uavs_mon_df_list = []

    rx_df_list = [gcs_rx_df, gw_rx_df] + uavs_rx_df_list
    tx_df_list = [gcs_tx_df, gw_tx_df] + uavs_tx_df_list
    pd_df_list = [gcs_pd_df, gw_pd_df] + uavs_pd_df_list
    mon_df_list = [gcs_mon_df, gw_mon_df] + uavs_mon_df_list

    # UNCOMMENT BELOW IF RSSI DATA WILL BE USED
    for rx_df in rx_df_list:
        rx_df["RSSI"] = rssi_to_np(rx_df["RSSI"])
    for pd_df in pd_df_list:
        pd_df["RSSI"] = rssi_to_np(pd_df["RSSI"])
    for mon_df in mon_df_list:
        if mon_df is not None:
            mon_df["RSSI"] = rssi_to_np(mon_df["RSSI"])

    return rx_df_list, tx_df_list, pd_df_list, mon_df_list

# def process_sim_data(sim_root_path, scenario, delay_threshold):
#     '''
#     VERSION 2
#     '''
#     # Dataframes to store UL & DL raw data
#     dl_df = pd.DataFrame(columns = ['RxTime','TxTime','Packet_Name','Bytes','RSSI','U2G_SINR','U2U_SINR','U2G_BER','U2U_BER','Hop_Count','Throughput',
#                                     'Delay','Queueing_Time','Backoff_Time','U2G_Distance','Height','Inter_UAV_Distance','Num_Members','Sending_Interval',
#                                     'Incorrectly_Rcvd','Queue_Overflow','Interface_Down','Number_Dropped','Packet_State']) # Downlink dataframe
#     ul_df = pd.DataFrame(columns = ['RxTime','TxTime','Packet_Name','Bytes','RSSI','U2G_SINR','U2U_SINR','U2G_BER','U2U_BER','Hop_Count','Throughput',
#                                     'Delay','Queueing_Time','Backoff_Time','U2G_Distance','Height','Inter_UAV_Distance','Num_Members','Sending_Interval',
#                                     'Incorrectly_Rcvd','Queue_Overflow','Interface_Down','Number_Dropped','Packet_State']) # Downlink dataframe

#     # For each scenario, extract the UL and DL raw data
#     NP = 10000 # The number of packets set in the simulation for each 100m (refer to OMNeT++ ini sim file)
#     # scenario = "NumMember-5_InterUAVDistance-5_Height-84_Distance-0_PacketSize-249_SendingRate-425"
#     scenario_files = glob.glob(sim_root_path + "/{}_*.csv".format(scenario)) # Get list of csv files belonging to this scenario
#     scenario_params = scenario.split('_')
#     num_member = int(scenario_params[0].split('-')[-1])
#     inter_uav_distance = int(scenario_params[1].split('-')[-1])
#     height = int(scenario_params[2].split('-')[-1]) 
#     sending_interval = int(scenario_params[5].split('-')[-1])
#     rx_df_list, tx_df_list, pd_df_list, mon_df_list = compile_micro_sim_data(scenario_files)
        
    
    
#     return dl_df, ul_df

In [4]:
print(rx_df_list[1].dtypes)

RxTime           float64
TxTime           float64
Packet_Name       object
Bytes              int64
RSSI             float64
U2G_SINR         float64
U2U_SINR         float64
U2G_BER          float64
U2U_BER          float64
Src_Addr          object
Dest_Addr         object
Hop_Count          int64
Delay            float64
Queueing_Time    float64
Backoff_Time     float64
U2G_Distance     float64
Unnamed: 16      float64
dtype: object


This section is to compare different modulation schemes

In [30]:
# Let's get the data
# sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np1000_BPSK_6-5Mbps"
# sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np1000_16QAM_39Mbps"
sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np1000_64QAM_65Mbps"
# scenario = "NumMember-3_InterUAVDistance-5_Height-24_Distance-0_PacketSize-24_SendingRate-40"
# scenario = "NumMember-3_InterUAVDistance-5_Height-50_Distance-0_PacketSize-473_SendingRate-425"
scenario = "NumMember-3_InterUAVDistance-5_Height-108_Distance-0_PacketSize-921_SendingRate-809"
scenario_files = glob.glob(sim_root_path + "/{}_*.csv".format(scenario)) # Get list of csv files belonging to this scenario
rx_df_list, tx_df_list, pd_df_list, mon_df_list = compile_micro_sim_data(scenario_files)
scenario_params = scenario.split('_')
height = int(scenario_params[2].split('-')[-1]) 
for i in range(len(rx_df_list)):
    # df["Horizontal_Distance"] = math.sqrt(df["U2G_Distance"]**2 - height**2)
    rx_df_list[i]["Horizontal_Distance"] = rx_df_list[i].apply(lambda x: math.sqrt(x["U2G_Distance"]**2 - height**2), axis=1)

In [31]:
# Get DOWNLINK RELIABILITY
delay_threshold = 0.04 # Delay threshold in sec for reliability
distance_classes = [0,100,200,300,400,500,600,700,800,900,1000]
reliability_list = []
num_tx_per_class = int(len(tx_df_list[0].index) / (len(distance_classes)-1)) # Assuming that the number of packets sent spilt evenly among the distance classes
# For each distance class, get the downlink reliability
for i in range(len(distance_classes)-1):
    dist_lower_bnd = distance_classes[i]
    dist_upper_bnd = distance_classes[i+1]
    # For each UAV, get the number of reliable packets within this distance class
    num_reliable = 0
    for j in range(1,len(rx_df_list)):
        uav_df = rx_df_list[j]
        reliable_packets_df = uav_df.loc[(uav_df["Horizontal_Distance"] >= dist_lower_bnd) & (uav_df["Horizontal_Distance"] < dist_upper_bnd) & (uav_df["Delay"] <= delay_threshold)]
        num_reliable += len(reliable_packets_df.index)
    # Record reliability at each dist class
    reliability_list.append(num_reliable / num_tx_per_class)

print(reliability_list)
    

[0.069, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [34]:
# Get UPLINK RELIABILITY (From UAV to GCS)
delay_threshold = 0.04 # Delay threshold in sec for reliability
distance_classes = [0,100,200,300,400,500,600,700,800,900,1000]
reliability_list = []
num_tx = 0
for j in range(1,len(tx_df_list)):
    num_tx += len(tx_df_list[j].index)
num_tx_per_class = int(num_tx / (len(distance_classes)-1)) # Assuming that the number of packets sent spilt evenly among the distance classes
gcs_rx_df = rx_df_list[0]
# For each distance class, get the uplink reliability
for i in range(len(distance_classes)-1):
    num_reliable = 0
    dist_lower_bnd = distance_classes[i]
    dist_upper_bnd = distance_classes[i+1]
    reliable_packets_df = gcs_rx_df.loc[(gcs_rx_df["Horizontal_Distance"] >= dist_lower_bnd) & (gcs_rx_df["Horizontal_Distance"] < dist_upper_bnd) & (gcs_rx_df["Delay"] <= delay_threshold)]
    num_reliable += len(reliable_packets_df.index)
    # Record reliability at each dist class
    reliability_list.append(num_reliable / num_tx_per_class)

print(reliability_list)

[0.09236145781328008, 0.0007488766849725411, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


This section is to test the convergence of number of samples (packets)

In [20]:
# Let's get the data
# sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np1000_BPSK_6-5Mbps"
# sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np10000_BPSK_6-5Mbps"
sim_root_path = "/home/research-student/omnetpp_sim_results/Test_Np100000_BPSK_6-5Mbps"
# scenario = "NumMember-3_InterUAVDistance-5_Height-24_Distance-0_PacketSize-24_SendingRate-40"
# scenario = "NumMember-3_InterUAVDistance-5_Height-50_Distance-0_PacketSize-473_SendingRate-425"
scenario = "NumMember-3_InterUAVDistance-5_Height-108_Distance-0_PacketSize-921_SendingRate-809"
scenario_files = glob.glob(sim_root_path + "/{}_*.csv".format(scenario)) # Get list of csv files belonging to this scenario
rx_df_list, tx_df_list, pd_df_list, mon_df_list = compile_micro_sim_data(scenario_files)
scenario_params = scenario.split('_')
height = int(scenario_params[2].split('-')[-1]) 
for i in range(len(rx_df_list)):
    # df["Horizontal_Distance"] = math.sqrt(df["U2G_Distance"]**2 - height**2)
    rx_df_list[i]["Horizontal_Distance"] = rx_df_list[i].apply(lambda x: math.sqrt(x["U2G_Distance"]**2 - height**2), axis=1)

In [21]:
# Get DOWNLINK RELIABILITY
delay_threshold = 0.04 # Delay threshold in sec for reliability
distance_classes = [0,100,200,300,400,500,600,700,800,900,1000]
reliability_list = []
num_tx_per_class = int(len(tx_df_list[0].index) / (len(distance_classes)-1)) # Assuming that the number of packets sent spilt evenly among the distance classes
# For each distance class, get the downlink reliability
for i in range(len(distance_classes)-1):
    dist_lower_bnd = distance_classes[i]
    dist_upper_bnd = distance_classes[i+1]
    # For each UAV, get the number of reliable packets within this distance class
    num_reliable = 0
    for j in range(1,len(rx_df_list)):
        uav_df = rx_df_list[j]
        reliable_packets_df = uav_df.loc[(uav_df["Horizontal_Distance"] >= dist_lower_bnd) & (uav_df["Horizontal_Distance"] < dist_upper_bnd) & (uav_df["Delay"] <= delay_threshold)]
        num_reliable += len(reliable_packets_df.index)
    # Record reliability at each dist class
    reliability_list.append(num_reliable / num_tx_per_class)

print(reliability_list)

[0.9995700343972482, 0.9998500119990401, 0.9808415326773858, 0.603561715062795, 0.18851491880649549, 0.07289416846652268, 0.03709703223742101, 0.022548196144308454, 0.01573874090072794, 0.011779057675385968]


In [22]:
# Get UPLINK RELIABILITY (From UAV to GCS)
delay_threshold = 0.04 # Delay threshold in sec for reliability
distance_classes = [0,100,200,300,400,500,600,700,800,900,1000]
reliability_list = []
num_tx = 0
for j in range(1,len(tx_df_list)):
    num_tx += len(tx_df_list[j].index)
num_tx_per_class = int(num_tx / (len(distance_classes)-1)) # Assuming that the number of packets sent spilt evenly among the distance classes
gcs_rx_df = rx_df_list[0]
# For each distance class, get the uplink reliability
for i in range(len(distance_classes)-1):
    num_reliable = 0
    dist_lower_bnd = distance_classes[i]
    dist_upper_bnd = distance_classes[i+1]
    reliable_packets_df = gcs_rx_df.loc[(gcs_rx_df["Horizontal_Distance"] >= dist_lower_bnd) & (gcs_rx_df["Horizontal_Distance"] < dist_upper_bnd) & (gcs_rx_df["Delay"] <= delay_threshold)]
    num_reliable += len(reliable_packets_df.index)
    # Record reliability at each dist class
    reliability_list.append(num_reliable / num_tx_per_class)

print(reliability_list)

[1.0000225001687513, 0.9999874999062492, 0.9845948844616335, 0.6634249756873176, 0.23233174248806865, 0.08596314472358543, 0.041137808533564, 0.02436518273887054, 0.016340122550919132, 0.011670087525656443]
